In [1]:
from numpy.random import seed
seed(1)

In [2]:
# Dependencies
import numpy as np
import pandas as pd

In [3]:
import tensorflow
tensorflow.keras.__version__

'2.6.0'

In [4]:
employee = pd.read_csv('employee.csv')
employee.head()

,employee_id,is_married,marital_status_id,employee_status_id,department_id,perf_score_id,is_diversity_job_fair,employee_salary,is_active,is_terminated,...,is_underrep_gender,is_citizen,is_hispanic_latino,is_underrep_race_eth,employee_status,survey_engagement_score,survey_emp_satisfaction_score,count_special_projects,count_days_late_past_30_days,count_absences
0,10184,0,0,1,5,3,0,65288,1,0,...,1,1,0,0,Active,3.19,3,0,0,9
1,10104,0,0,1,5,3,0,66738,1,0,...,0,1,0,0,Active,4.53,5,0,0,5
2,10047,1,1,3,5,3,0,50428,0,1,...,1,1,0,1,Voluntarily Terminated,5.00,3,0,0,11
3,10044,1,1,3,3,3,0,75281,0,1,...,1,1,0,0,Voluntarily Terminated,5.00,3,5,0,11
4,10238,1,1,1,2,3,1,63000,1,0,...,0,1,0,1,Active,4.50,2,6,0,14


In [5]:
X = employee.drop("employee_status", axis=1)
y = employee["employee_status"]
print(X.shape, y.shape)

(311, 21) (311,)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [8]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [10]:
# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [12]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=21))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [13]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               2200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 12,603
Trainable params: 12,603
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=20,
    shuffle=True,
    verbose=2
)

Epoch 1/20
8/8 - 0s - loss: 0.8701 - accuracy: 0.6567
Epoch 2/20
8/8 - 0s - loss: 0.5725 - accuracy: 0.6824
Epoch 3/20
8/8 - 0s - loss: 0.3944 - accuracy: 0.9099
Epoch 4/20
8/8 - 0s - loss: 0.2766 - accuracy: 0.9485
Epoch 5/20
8/8 - 0s - loss: 0.2029 - accuracy: 0.9485
Epoch 6/20
8/8 - 0s - loss: 0.1619 - accuracy: 0.9485
Epoch 7/20
8/8 - 0s - loss: 0.1365 - accuracy: 0.9485
Epoch 8/20
8/8 - 0s - loss: 0.1191 - accuracy: 0.9485
Epoch 9/20
8/8 - 0s - loss: 0.1033 - accuracy: 0.9485
Epoch 10/20
8/8 - 0s - loss: 0.0910 - accuracy: 0.9571
Epoch 11/20
8/8 - 0s - loss: 0.0802 - accuracy: 0.9700
Epoch 12/20
8/8 - 0s - loss: 0.0698 - accuracy: 0.9742
Epoch 13/20
8/8 - 0s - loss: 0.0638 - accuracy: 0.9914
Epoch 14/20
8/8 - 0s - loss: 0.0556 - accuracy: 0.9785
Epoch 15/20
8/8 - 0s - loss: 0.0491 - accuracy: 0.9828
Epoch 16/20
8/8 - 0s - loss: 0.0426 - accuracy: 0.9914
Epoch 17/20
8/8 - 0s - loss: 0.0379 - accuracy: 0.9914
Epoch 18/20
8/8 - 0s - loss: 0.0345 - accuracy: 0.9871
Epoch 19/20
8/8 - 0

In [16]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

3/3 - 0s - loss: 0.0712 - accuracy: 0.9744
Normal Neural Network - Loss: 0.07122822850942612, Accuracy: 0.9743589758872986


In [17]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

AttributeError: 'Sequential' object has no attribute 'predict_classes'

In [ ]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")